In [1]:
import pandas as pd
import random
import json
import re

In [6]:
import pandas as pd
import random

# Load datasets
eval_df = pd.read_csv('data/eval.csv')
slang_df = pd.read_csv('data/slang_data.csv')

# Normalize slang keys
eval_df['slang_norm'] = eval_df['slang'].astype(str).str.strip().str.lower()
slang_df['slang_norm'] = slang_df['slang'].astype(str).str.strip().str.lower()

# Deduplicate slang definitions
slang_defs = slang_df[['slang_norm', 'definition']].drop_duplicates()

# Build dict: slang -> definition
slang_to_def = {row['slang_norm']: row['definition'] for _, row in slang_defs.iterrows()}

# All definitions for distractor pool
all_defs = list(slang_to_def.values())

rows = []

# Template patterns for paraphrases
templates = [
    "Here, {} probably means {}.",
    "It suggests that {} is like {}.",
    "It implies that {} could be {}.",
    "In other words, {} likely means {}."
]

for _, row in eval_df.iterrows():

    slang = row['slang_norm']
    examples = [row['example_1'], row['example_2'], row['example_3']]

    # Correct definition
    true_def = slang_to_def[slang]

    # Sample distractor definitions
    distractor_defs = random.sample(
        [d for d in all_defs if d != true_def], 
        3
    )

    for ex in examples:
        sentence = ex

        # Build correct paraphrase
        correct = random.choice(templates).format(slang, true_def.lower())

        # Build distractor paraphrases
        distractors = [
            random.choice(templates).format(slang, d.lower())
            for d in distractor_defs
        ]

        # Randomize option order
        options = [('A', correct)] + [('B', distractors[0]), ('C', distractors[1]), ('D', distractors[2])]
        
        # Shuffle but track correct label
        random.shuffle(options)
        labels = ['A', 'B', 'C', 'D']
        option_map = {labels[i]: options[i][1] for i in range(4)}
        correct_label = next(label for label, text in option_map.items() if text == correct)

        # Add row
        rows.append({
            'sentence': sentence,
            'option_A': options[0][1],
            'option_B': options[1][1],
            'option_C': options[2][1],
            'option_D': options[3][1],
            'correct': correct_label
        })

# Convert to DataFrame and save
reverse_df = pd.DataFrame(rows)
reverse_df.to_csv('data/eval_reverse.csv', index=False)

reverse_df.head()


,sentence,option_A,option_B,option_C,option_D,correct
0,His drip was so clean that even the photograph...,It implies that drip could be stylish or fashi...,It implies that drip could be refers to someth...,It suggests that drip is like used to express ...,"Here, drip probably means influence or power..",A
1,She upgraded her drip after finding that thrif...,"Here, drip probably means refers to something ...",It implies that drip could be used to express ...,"Here, drip probably means influence or power..","In other words, drip likely means stylish or f...",D
2,They walked in together with coordinated drip ...,It suggests that drip is like influence or pow...,It implies that drip could be refers to someth...,"Here, drip probably means stylish or fashionab...",It suggests that drip is like used to express ...,C
3,Her boujee taste had her rejecting every resta...,It suggests that boujee is like car..,"Here, boujee probably means used to warn someo...","Here, boujee probably means luxurious or fancy..",It implies that boujee could be suspicious or ...,C
4,He acted boujee all night after getting a sing...,It suggests that boujee is like luxurious or f...,It implies that boujee could be used to warn s...,It suggests that boujee is like suspicious or ...,It implies that boujee could be car..,A


In [10]:
import pandas as pd
import json
import re

# Load training source
slang_df = pd.read_csv('data/slang_data.csv')

# Normalize (in case there are stray spaces / case issues)
slang_df['slang_norm'] = slang_df['slang'].astype(str).str.strip().str.lower()

def clean_def(d: str) -> str:
    d = str(d).strip()
    # remove trailing punctuation like "." or "..."
    d = re.sub(r'[\.!\?]+\s*$', '', d)
    return d

records = []

for _, row in slang_df.iterrows():
    example = row['example']
    definition = row['definition']

    if not isinstance(example, str) or not isinstance(definition, str):
        continue

    def_clean = clean_def(definition)

    prompt = (
        "You will be given a sentence that contains a modern slang term.\n"
        "Explain what the sentence means in standard English, focusing on the slang.\n\n"
        f"Sentence: \"{example}\"\n"
        "Meaning:"
    )

    # simple completion based on the definition
    completion = " " + def_clean.lower() + "."

    records.append({
        "prompt": prompt,
        "completion": completion
    })

out_path = 'data/finetune_reverse.jsonl'
with open(out_path, 'w', encoding='utf-8') as f:
    for rec in records:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("Wrote", len(records), "examples to", out_path)


Wrote 547 examples to data/finetune_reverse.jsonl


In [11]:
with open(out_path, 'r', encoding='utf-8') as f:
    records = f.readlines()
    records = [json.loads(line) for line in records]
    
print("Loaded", len(records), "examples from", out_path)

Loaded 547 examples from data/finetune_reverse.jsonl


In [15]:
print(records[0]['prompt'], records[0]['completion'])

You will be given a sentence that contains a modern slang term.
Explain what the sentence means in standard English, focusing on the slang.

Sentence: "His drip today is on point."
Meaning:  stylish or fashionable, especially in clothing.


In [2]:
import pandas as pd
import json
import random
import re

# ---------- config ----------
SLANG_PATH = "data/slang_data.csv"
FINETUNE_REVERSE_MCQ_JSONL = "data/finetune_reverse_mcq.jsonl"
random.seed(42)
# ----------------------------

slang_df = pd.read_csv(SLANG_PATH)
slang_df["slang_norm"] = slang_df["slang"].astype(str).str.strip().str.lower()

# build mapping slang_norm -> definition
defs_df = slang_df[["slang_norm", "definition"]].drop_duplicates()
slang_to_def = {
    row["slang_norm"]: str(row["definition"]).strip()
    for _, row in defs_df.iterrows()
}

all_defs = list(set(slang_to_def.values()))

def clean_def(d: str) -> str:
    d = str(d).strip()
    d = re.sub(r"[\.!\?]+\s*$", "", d)
    return d

def make_paraphrase(slang: str, def_text: str) -> str:
    base = clean_def(def_text).lower()
    templates = [
        "Here, {} probably means {}.",
        "It suggests that {} is like {}.",
        "It implies that {} could be {}.",
        "In other words, {} likely means {}."
    ]
    return random.choice(templates).format(slang, base)

records = []

for _, row in slang_df.iterrows():
    slang = row["slang"]
    slang_norm = row["slang_norm"]
    example = row["example"]

    if not isinstance(example, str):
        continue
    if slang_norm not in slang_to_def:
        continue

    true_def = slang_to_def[slang_norm]
    true_paraphrase = make_paraphrase(slang_norm,true_def)

    # choose 3 *different* definitions for distractors
    distractor_pool = [d for d in all_defs if d != true_def]
    if len(distractor_pool) < 3:
        continue
    distractor_defs = random.sample(distractor_pool, 3)
    distractor_paraphrases = [make_paraphrase(slang_norm, d) for d in distractor_defs]

    # assign to A/B/C/D and then shuffle positions
    letters = ["A", "B", "C", "D"]
    opts = [true_paraphrase] + distractor_paraphrases
    positions = list(range(4))
    random.shuffle(positions)

    shuffled_opts = [opts[i] for i in positions]
    correct_index = shuffled_opts.index(true_paraphrase)
    correct_letter = letters[correct_index]

    prompt = (
        "You will be given a sentence that contains a modern slang term and four possible interpretations.\n"
        "Choose the option that best explains the meaning of the sentence in standard English.\n\n"
        f"Sentence: \"{example}\"\n\n"
        "Options:\n"
        f"A) {shuffled_opts[0]}\n"
        f"B) {shuffled_opts[1]}\n"
        f"C) {shuffled_opts[2]}\n"
        f"D) {shuffled_opts[3]}\n\n"
        "Answer with just the letter."
    )

    # leading space so completion tends to be a separate token
    completion = " " + correct_letter

    records.append({"prompt": prompt, "completion": completion})

print("Total reverse-MCQ finetune examples:", len(records))

with open(FINETUNE_REVERSE_MCQ_JSONL, "w", encoding="utf-8") as f:
    for rec in records:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("Wrote:", FINETUNE_REVERSE_MCQ_JSONL)


Total reverse-MCQ finetune examples: 547
Wrote: data/finetune_reverse_mcq.jsonl


In [3]:
with open(FINETUNE_REVERSE_MCQ_JSONL, "r", encoding="utf-8") as f:
    records = f.readlines()
    records = [json.loads(line) for line in records]

record = random.choice(records)
print(record["prompt"])
print(record["completion"])

You will be given a sentence that contains a modern slang term and four possible interpretations.
Choose the option that best explains the meaning of the sentence in standard English.

Sentence: "I just spilled coffee all over my desk, LOL."

Options:
A) It implies that lol could be a subtle or indirect insult.
B) In other words, lol likely means a common expression of amusement, often used lightly.
C) In other words, lol likely means refers to a romantic partner who is male.
D) Here, lol probably means stylish or fashionable, especially in clothing.

Answer with just the letter.
 B


In [4]:
import pandas as pd
import json
import re

EVAL_PATH = "data/eval.csv"
SLANG_PATH = "data/slang_data.csv"
OUT_PATH  = "data/eval_reverse_rl.jsonl"

eval_df = pd.read_csv(EVAL_PATH)
slang_df = pd.read_csv(SLANG_PATH)

# normalize slang keys
eval_df["slang_norm"]  = eval_df["slang"].astype(str).str.strip().str.lower()
slang_df["slang_norm"] = slang_df["slang"].astype(str).str.strip().str.lower()

# build slang_norm -> definition mapping (take first per slang)
defs = (
    slang_df[["slang_norm", "definition"]]
    .drop_duplicates()
    .groupby("slang_norm")
    .first()
    .reset_index()
)

slang_to_def = {
    row["slang_norm"]: str(row["definition"]).strip()
    for _, row in defs.iterrows()
}

def clean_def(d: str) -> str:
    d = str(d).strip()
    d = re.sub(r"[\.!\?]+\s*$", "", d)
    return d

records = []

for _, row in eval_df.iterrows():
    slang_norm = row["slang_norm"]
    if slang_norm not in slang_to_def:
        continue

    definition = clean_def(slang_to_def[slang_norm])

    for col in ["example_1", "example_2", "example_3"]:
        sent = row[col]
        if not isinstance(sent, str):
            continue

        prompt = (
            "You will be given a sentence that contains a modern slang term.\n"
            "Explain what the sentence means in standard English, focusing on the slang.\n\n"
            f"Sentence: \"{sent}\"\n"
            "Meaning:"
        )

        target = definition  # gold meaning to compare against

        records.append({"prompt": prompt, "target": target})

print("Eval examples:", len(records))

with open(OUT_PATH, "w", encoding="utf-8") as f:
    for rec in records:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("Wrote:", OUT_PATH)


Eval examples: 690
Wrote: data/eval_reverse_rl.jsonl
